# invoke with an image

In [1]:
import io 
import sagemaker
import boto3
import json
from sagemaker import get_execution_role

import base64


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:

role = get_execution_role()
region = boto3.Session().region_name
print(role, region)

sagemaker_session = sagemaker.session.Session()
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")




arn:aws:iam::396608815783:role/service-role/AmazonSageMaker-ExecutionRole-20250207T102425 us-west-2


In [3]:
# SageMaker Endpoint 이름
# endpoint_name = sagemaker.utils.name_from_base("QWEN-2-5-7B-G5")
# endpoint_name = "QWEN-2-5-7B-G5-2025-08-12-03-05-19-368"


all_endpoint = sm_client.list_endpoints()
print(all_endpoint,"\n")
endpoint_name = all_endpoint['Endpoints'][0]['EndpointName']
print(" - endpoint_name : ", endpoint_name)

image_path = "images/test001.png"

{'Endpoints': [{'EndpointName': 'gemma3-s3-vllm-async-2025-08-14-02-10-03-005', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/gemma3-s3-vllm-async-2025-08-14-02-10-03-005', 'CreationTime': datetime.datetime(2025, 8, 14, 2, 10, 5, 750000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 8, 14, 2, 19, 17, 309000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'gemma3-s3-vllm-async-2025-08-14-01-16-07-849', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/gemma3-s3-vllm-async-2025-08-14-01-16-07-849', 'CreationTime': datetime.datetime(2025, 8, 14, 1, 16, 9, 400000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 8, 14, 1, 25, 31, 472000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'QWEN-2-5-7B-G5-2025-08-12-03-05-19-368', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/QWEN-2-5-7B-G5-2025-08-12-03-05-19-368', 'CreationTime': datetime.datetime(2025, 8, 12, 3, 5, 

In [4]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)


In [5]:
# 1. 이미지 파일을 base64로 인코딩
with open(image_path, "rb") as f:
    image_b64 = base64.b64encode(f.read()).decode("utf-8")


payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                # {"type": "image", "data": image_b64},
                # {"type": "text", "text": "이 이미지는 무엇인가요?"}
                # {"type": "text", "text": " Ai Agent 무엇인가요?"}
                # {"type": "text", "text": "이 이미지는 무엇인가요?"}
                {"type": "text", "text": "안녕하세요."},
                # {"type": "image", "image_url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
                # {"type": "image", "data": image_b64},
                
                
            ]
        }
    ],
    "max_tokens": 256,
    "temperature": 0.8
}

payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                # {"type": "text", "text": "hello."},
                
                {"type": "image", "image_base64": image_b64},
                {"type": "text", "text": "이 이미지의 내용을 설명해 주세요."},
            ]
        }
    ]
}

single_image_payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG",
                    }
                },
                {
                    "type": "text",
                    "text": "이미지에 대해서 설명해줘.",
                },
                
            ]
        }
    ],
    "max_tokens": 1024,
    "temperature": 0.6,
    "top_p": 0.9,
}


In [6]:
# JSON 문자열로 변환
# payload_json = json.dumps(payload)

# 엔드포인트에 요청 보내기
# response = predictor.predict(data=payload_json)
response = predictor.predict(single_image_payload)

In [7]:
response

{'id': 'chatcmpl-d3b75864c81b4112b527c2c56e29fcf0',
 'object': 'chat.completion',
 'created': 1755141052,
 'model': 'lmi',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'reasoning_content': None,
    'content': '이미지에는 다음과 같은 내용이 담겨 있습니다:\n\n*   **손:** 밝은 갈색 피부의 손이 보입니다. 손바닥이 위를 향하고 있으며, 손가락 사이에는 다양한 색깔의 캔디가 놓여 있습니다.\n*   **캔디:** 손바닥에는 다음과 같은 캔디들이 놓여 있습니다.\n    *   **파란색 캔디:** 파란색 바탕에 녹색 잎 모양의 그림이 그려져 있습니다.\n    *   **청록색 캔디:** 청록색 캔디에는 검은색 잎 모양의 그림이 그려져 있습니다.\n    *   **주황색 캔디:** 주황색 캔디에는 녹색 잎 모양의 그림이 그려져 있습니다.\n    *   **초록색 캔디:** 초록색 캔디에는 검은색 잎 모양의 그림이 그려져 있습니다.\n*   **배경:** 손과 캔디는 밝은 나무 질감의 테이블 위에 놓여 있습니다. 테이블 뒤로는 파란색 옷을 입은 사람이 보입니다.\n\n전체적으로, 손에 다양한 색깔과 디자인의 캔디들이 놓여 있는 모습이 담겨 있습니다.',
    'tool_calls': []},
   'logprobs': None,
   'finish_reason': 'stop',
   'stop_reason': 106}],
 'usage': {'prompt_tokens': 275,
  'total_tokens': 541,
  'completion_tokens': 266,
  'prompt_tokens_details': None},
 'prompt_logprobs': None}

In [8]:
response['choices'][0]['message']

{'role': 'assistant',
 'reasoning_content': None,
 'content': '이미지에는 다음과 같은 내용이 담겨 있습니다:\n\n*   **손:** 밝은 갈색 피부의 손이 보입니다. 손바닥이 위를 향하고 있으며, 손가락 사이에는 다양한 색깔의 캔디가 놓여 있습니다.\n*   **캔디:** 손바닥에는 다음과 같은 캔디들이 놓여 있습니다.\n    *   **파란색 캔디:** 파란색 바탕에 녹색 잎 모양의 그림이 그려져 있습니다.\n    *   **청록색 캔디:** 청록색 캔디에는 검은색 잎 모양의 그림이 그려져 있습니다.\n    *   **주황색 캔디:** 주황색 캔디에는 녹색 잎 모양의 그림이 그려져 있습니다.\n    *   **초록색 캔디:** 초록색 캔디에는 검은색 잎 모양의 그림이 그려져 있습니다.\n*   **배경:** 손과 캔디는 밝은 나무 질감의 테이블 위에 놓여 있습니다. 테이블 뒤로는 파란색 옷을 입은 사람이 보입니다.\n\n전체적으로, 손에 다양한 색깔과 디자인의 캔디들이 놓여 있는 모습이 담겨 있습니다.',
 'tool_calls': []}